In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Read the Data

In [3]:
data = pd.read_csv("train.csv" , parse_dates=['Tarih'] , index_col='Tarih')
# data.set_index('Tarih' , drop= True , inplace=True)
data.head()

,Dağıtılan Enerji (MWh)
Tarih,
2018-01-01 00:00:00,1593.944216
2018-01-01 01:00:00,1513.933887
2018-01-01 02:00:00,1402.612637
2018-01-01 03:00:00,1278.527266
2018-01-01 04:00:00,1220.697701


In [4]:
future_data = pd.read_csv("future.csv", parse_dates=['Tarih'] , index_col= 'Tarih').drop('Unnamed: 0' , axis = 1)
future_data.head()

,Dağıtılan Enerji (MWh)
Tarih,
2022-08-01 00:00:00,NaN
2022-08-01 01:00:00,NaN
2022-08-01 02:00:00,NaN
2022-08-01 03:00:00,NaN
2022-08-01 04:00:00,NaN


In [5]:
med = pd.read_csv('med.csv')
med['Tarih'] = pd.to_datetime(med['Tarih'])
med.head()

,Tarih
0,2019-12-23
1,2019-06-12
2,2019-01-25
3,2019-09-25
4,2019-03-29


# Describe the Data

In [6]:
data.shape

(40152, 1)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40152 entries, 2018-01-01 00:00:00 to 2022-07-31 23:00:00
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Dağıtılan Enerji (MWh)  40152 non-null  float64
dtypes: float64(1)
memory usage: 627.4 KB


In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Dağıtılan Enerji (MWh),40152.0,1836.805287,426.066085,870.18328,1499.165048,1813.409221,2129.407998,3633.105297


# Plot the Data

# Train | Test Split

In [ ]:
# # split into train and test sets

# n_train_time = 1460*24
# train = values[:n_train_time, :]
# test = values[n_train_time:, :]

In [9]:
# X_data = data.copy()
X_test_temp = data.loc[data.index > '2022-03-23 23:00:00'].copy()
X_train_temp = data.loc[data.index <= '2022-03-23 23:00:00'].copy()

# X_val_temp = X_train_temp[-144:]
# X_train_temp = X_train_temp[:-144]

In [10]:
# # split into input and outputs
# X_train, y_train = train[:, :], train[:, 0]
# X_test, y_test = test[:, :], test[:, 0]

In [11]:
# print(f"X_data Shape : {X_data.shape}")
print(f"Full data shape : {data.shape}")
print(f"X_train Shape : {X_train_temp.shape}")
print(f"X_test Shape : {X_test_temp.shape}")
# print(f"X_val Shape : {X_val_temp.shape}")
# print(f"X Shape : {X.shape}")

Full data shape : (40152, 1)
X_train Shape : (37032, 1)
X_test Shape : (3120, 1)


# Scaling

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(data)
X_train_scaled = scaler.transform(X_train_temp)
X_test_scaled = scaler.transform(X_test_temp)

# Reshape and Sequence

Buraya dikkat

In [ ]:
def packager(data_scaled , train_scaled, X_test_scaled, window_size=24):
    X_data , y_data , X_train, y_train, X_test, y_test = [], [], [], [], [], []
    
    
    for i in range(window_size, len(data_scaled)):
        X_data.append(data_scaled[i-window_size:i, 0])
        y_data.append(data_scaled[i, 0])

    for i in range(window_size, len(train_scaled)):
        X_train.append(train_scaled[i-window_size:i, 0])
        y_train.append(train_scaled[i, 0])

    # for i in range(window_size, len(val_scaled)):
    #     X_val.append(val_scaled[i-window_size:i, 0])
    #     y_val.append(val_scaled[i, 0])

    for i in range(window_size, len(X_test_scaled)):
        X_test.append(X_test_scaled[i-window_size:i, 0])
        y_test.append(X_test_scaled[i, 0])

    X_data, y_data = np.array(X_data), np.array(y_data)
    X_train, y_train = np.array(X_train), np.array(y_train)
    # X_val, y_val = np.array(X_val), np.array(y_val)
    X_test, y_test = np.array(X_test), np.array(y_test)

    X_data = np.reshape(X_data, (X_data.shape[0], X_train.shape[1], 1))
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    # X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # X = np.concatenate((X_train, X_test), axis=0)
    # y = np.concatenate((y_train, y_test), axis=0)

    return X_data , y_data , X_train, y_train, X_test, y_test

In [ ]:
X_data , y_data , X_train, y_train, X_test, y_test = packager(data_scaled , X_train_scaled, X_test_scaled, 24)

In [ ]:
print(f"X_data Shape : {X_data.shape}")
print(f"y_data Shape : {y_data.shape}\n\n")
print(f"X_train Shape : {X_train.shape}")
print(f"y_train Shape : {y_train.shape}\n\n")
print(f"X_test Shape : {X_test.shape}")
print(f"y_test Shape : {y_test.shape}")

# Final Model

In [ ]:
import tensorflow as tf

# Seed ayarlamak
seed_value = 61
tf.keras.utils.set_random_seed(seed_value)

# Manual Modelling

In [ ]:
import tensorflow as tf

In [ ]:
X_train.shape

In [ ]:
# window_size = 24
# batch_size = 8

from keras.models import Sequential
from keras.layers import LSTM, Dense , BatchNormalization , Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
from keras.optimizers import Adam

# Model oluşturma
model = Sequential()
# model.add(LSTM(50, input_shape=(window_size, data.shape[1]) ,return_sequences = False))
model.add(LSTM(12, activation='tanh' , input_shape = (X_train.shape[1],X_train.shape[2]) ,return_sequences=True))
model.add(LSTM(8, activation='tanh' ,return_sequences=True))
# model.add(LSTM(30, return_sequences=False))
model.add(LSTM(4 , activation = 'tanh' , return_sequences=True))
model.add(LSTM(2 , activation = 'tanh' , return_sequences=False))
model.add(Dense(1))

opt = Adam(learning_rate = 0.0001)
model.compile(loss='mse', optimizer=opt)



reduce_lr = ReduceLROnPlateau(factor = 0.1 , patience=5)
early_stop = EarlyStopping(monitor='val_loss' , patience=15)
model_check = ModelCheckpoint(filepath=r'C:\Users\Emincan\Desktop\Gdz\gdz-competition\Checkpoints\Single_Target\\60-30-15(batch1-relu)\\model-{epoch:03d}-{val_loss:.5f}.h5', monitor="val_loss", verbose=1)

# # Modeli eğitme
# model.fit(X_train , y_train, epochs=30, batch_size = batch_size , validation_data=(X_test , y_test) , callbacks= [reduce_lr , early_stop , model_check] , shuffle=False )
history = model.fit(X_data, y_data, epochs=150, batch_size= 1, validation_split=0.05, callbacks= [reduce_lr , early_stop, model_check] , shuffle=False)


# Modeli değerlendirme
mse = model.evaluate(X_test , y_test)
print('Test MSE: %.3f' % mse)

# Import the Model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\Emincan\Desktop\Gdz\gdz-competition\\Checkpoints\Single_Target\\60-30-15(batch1-relu)\\model-005-0.00557.h5")

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper right')
# plt.show()

# Eval Metric

In [ ]:
from eval_metrics import eval_metrics

# Prediction

In [ ]:
data_scaled.shape

In [ ]:
data_scaled[-24:]

In [ ]:
data_last_seq = data_scaled[-24:]

In [ ]:
data_last_seq.shape

In [ ]:
model.predict(data_last_seq.reshape(1,24,1))

In [ ]:
# Elimizdeki verilerin son 24'ü bir sonraki tahmini yapmak için kullanılacak
last_window = data_last_seq.copy()

# 31 gün boyunca tahminler yapmak için bir boş tahmin dizisi oluşturun
predictions = []

for i in range(31*24):
    # Tahmin edilen değerleri ölçeklendirmek için son pencereyi yeniden şekillendirin
    last_window_reshaped = last_window.reshape(1, 24, 1)

    # Son pencereyi kullanarak bir tahmin yapın
    prediction = model.predict(last_window_reshaped , verbose = 0)[0][0]

    # Tahmini tahmin listesine ekleyin
    predictions.append(prediction)

    # Tahmin edilen değeri son pencerenin sonuna ekleyin
    last_window = np.append(last_window[1:], prediction)
    
# Tahminleri geri ölçeklendirin
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Tahminlerinizi burada kullanabilirsiniz

In [ ]:
predictions

## Visualize Predictions

In [ ]:
from eval_metrics import preds_plot

In [ ]:
future_data['Dağıtılan Enerji (MWh)'] = predictions
future_data

In [ ]:
target_data = data[data.index.month == 8]

In [ ]:
target_data_2018 = target_data[target_data.index.year == 2018]
target_data_2019 = target_data[target_data.index.year == 2019]
target_data_2020 = target_data[target_data.index.year == 2020]
target_data_2021 = target_data[target_data.index.year == 2021]

In [ ]:
data

In [ ]:
preds_plot(data , future_data , target_data )

In [ ]:
# Done